In [74]:
import random
#gcd們
#extended gcd
def xgcd(a, b):
    x0, x1, y0, y1 = 0, 1, 1, 0
    while a != 0:
        q, b, a = b // a, a, b % a
        y0, y1 = y1, y0 - q * y1
        x0, x1 = x1, x0 - q * x1
    return x0
#gcd
def gcd(a,b):
    if b == 0:
        return a
    else:
        return gcd(b,a%b)

#產生公私鑰
def gen_key(p, q):
    n = p * q
    phy_n = (p - 1) * (q - 1)   #phy_n
    e=0
    for i in range(2,phy_n): #找出盡量小的e(加速加密)
        e=i
        if gcd(e,phy_n)==1:
            break
    d=xgcd(e,phy_n)   #找出d
    if d<0:
        d+=phy_n
    print("n為：",n)
    print("phy_n為：",phy_n)
    print("e為：",e)
    print("d為：",d)
    # 返回：  公鑰   私鑰
    return  (n, e), (n, d)
#加密
def encrypt(m, pubkey):
    n = pubkey[0]
    e = pubkey[1]
    c = modExp(m, e, n)#m^e%n
    return c
# 解密 c是密文，解密為明文m
def decrypt(c, selfkey):
    n = selfkey[0]
    d = selfkey[1]
    m = modExp(c, d, n)#c^d%n
    return m



In [73]:
#蒙哥馬利冪模
def int2baseBinary(x):#將數字轉成binary形式並反過來由低到高位元
    binList = []
    while x != 0:
        binList.append(x%2)
        x = x >> 1
    return binList
def modExp(a, d, n):#計算a^d%n
    res = 1
    lt = int2baseBinary(d)
    for i in lt:
        if i:
            res = (res * a) % n
        a = (a * a) % n
    return res
    

In [72]:
#產生一個1024bit的隨機數頭尾皆為1   
def myrandom(n):
    #<1000的質數
    prime_lis=[]
    for i in range(2,1000):
        count=0
        for j in range(2,i):
            if (i % j) == 0:
                count+=1
                break
        if count==0:
            prime_lis.append(i)
    size=(len(prime_lis))
    check=1#用來判斷是否可被1000內質數整除
    while(check):
        ary='1'
        for i in range(n-2):
            a=random.randint(0,1)
            ary+=str(a)
        ary+='1'
        put=int(ary,2)#暫放的數字
        #判斷1000內質數
        count=0
        for i in range(size):
            if (put % prime_lis[i])==0:
                break
            else:
                count+=1
        if count==size:
            return put

In [71]:
#miller-rabin 驗證是否為質數
def miller_rabin(n):
    #n-1寫成(2^s)*d的形式
    s = 0
    d = n - 1
    while True:
        quotient, remainder = divmod(d, 2)
        if remainder == 1:
            break
        s += 1
        d = quotient
    assert(2 ** s * d == n - 1)
 
    #以a為底,n是否為合數
    def try_composite(a):
        if pow(a, d, n) == 1: #(a^d)%n
            return False
        for i in range(s):
            if pow(a, 2 ** i * d, n) == n - 1: #(a^((2^i)*d))%n
                return False
        return True # n為合數
    
    #每次取不同a,一次不符合即為合數
    for i in range(5):
        a = random.randrange(2, n)
        if try_composite(a):
            return False

    return True #很可能是質數

In [75]:
if __name__ == '__main__':
#step 1：產出p,q
    miller_p=1#檢查是否符合miller-rabin
    miller_q=1#檢查是否符合miller-rabin
    p=myrandom(1024)#產生一個不被1000內質數整除的隨機數節省時間
    q=myrandom(1024)#產生一個不被1000內質數整除的隨機數節省時間
    while(miller_p):
        if miller_rabin(p):
            miller_p=0
        else: #是合數
            p=myrandom(1024)#重新產生一個不被1000內質數整除的隨機數
    while(miller_q):
        if miller_rabin(q):
            miller_q=0
        else: #是合數
            q=myrandom(1024)#重新產生一個不被1000內質數整除的隨機數
#step 2：產出公私鑰
    print("p為：",p)
    print("q為：",q)
    public,private=gen_key(p,q)
    m=135620532045761028874519896765764416637997218983980438907459156366663406664656441068595521782504862606619086653659240596696402402223658759344712239254003849389312124894878052511782288923057497865141807540335743969274339825020706092092911760603349055915956098776876832482301157928322339296445443990454267563768
    cipher = encrypt(m, public)
    print("cipher為：",cipher)
    plain = decrypt(cipher, private)
    print("plain為：",plain)

p為： 137428996985351278495145292653726033378620093882784949228586640187071914383560554923078413469331098759466088239517614645333821760150047836725719814365886223482779700627901066931143163507478223620790268699490046040366564083760344024376506952260970244723468528831432388182236395837573408540953489432144667915597
q為： 118126798810279475643818597198764427057957816045263445723672435848002769127897695500363102127071754391677254791173895651772059501786810292603904611449690333134127450291819839891040927459383096718568522625339288844999137939366191921365509184793533411653513337407927581038602634758985949739019632914484788333553
n為： 16234047477587095049384132649124782392283872886536871319314311318097435440018595658449535987624258173421483642890342804517257378729314008770063394640468140090488117100385455001623863054821058548626240973594645093525334063101215360749082194191070245791650712646440147069897111163294975507525576793960421813511838590871195713405979279765978176863946009655419711193175943